In [1]:
import os
import numpy as np
from glob import glob
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
from scipy.io import loadmat

import warnings
warnings.filterwarnings('ignore')

In [2]:
#esm4_file = '/work/jyl/model_outputs/oar.gfdl.cmip6/ESM4/DECK/ESM4_piControl_D/gfdl.ncrc4-intel16-prod-openmp/history/ocean/00010101.ocean_annual_z.nc'
esm4_file = '/archive/oar.gfdl.cmip6/ESM4/DECK/ESM4_historical_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ts/annual/5yr/ocean_annual_z.2010-2014.thetao.nc'
esm4 = xr.open_dataset(esm4_file).isel(time=0)

In [3]:
cas_folder = '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/'
files = glob(cas_folder+'*esm4grid.mat')
files

['/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/Salt_woa_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/phosphate_woa_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/TCO2_glodap_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/nitrate_woa_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/silicate_woa_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/Temp_woa_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/O2_woa_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/PI_TCO2_glodap_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/TAlk_glodap_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/pH_glodap_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/dpco2_smoothed_monthclim_landschutzer_1995to2014_esm4grid.mat',
 '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/spco2_smoothe

In [5]:
varname_old = None

for file in files:
    print('Processing: ' + file)
    varname_file = file.replace(cas_folder, '').replace('.mat','')
    
    # a couple naming exceptions
    if varname_file == 'globcolor_soblend_mc_esm4grid':
        varname_file = 'globcolour_soblend_mc_esm4grid'
    if 'cas_mc' in varname_file:
        varname_old = varname_file
        varname_file = varname_file.replace('cas_','')
    
    varname_out = varname_file.replace('_esm4grid', '')
    print('Variable: ' + varname_out)
    
    # read data
    data = loadmat(file)
    data = data[varname_file]
    data.shape
    
    # write into xarray dataset format
    if data.shape[0]==35:
        da = xr.DataArray(data, coords={'z_l':esm4.z_l.values, 'yh':esm4.yh.values, 'xh':esm4.xh.values}, dims=['z_l','yh','xh'])
    if data.shape[0]==12:
        da = xr.DataArray(data, coords={'time':np.arange(1,13), 'yh':esm4.yh.values, 'xh':esm4.xh.values}, dims=['time','yh','xh'])
    ds = da.to_dataset(name=varname_out)
    
    # write into netcdf
    out_path='/work/jyl/proj/ESM4.2_COBALT/COBALT_diagnostics/Remapped_Data/'
    if varname_file.replace('mc_','cas_mc_') == varname_old:
        varname_file = varname_old
    filename=varname_file+'.nc'
    print('Writing out to: ' + out_path+filename)
    ds.to_netcdf(out_path+filename, mode='w')


Processing: /archive/cas/matlab_COBALT_diagnostics/Remapped_Data/Salt_woa_esm4grid.mat
Variable: Salt_woa
Writing out to: /work/jyl/proj/ESM4.2_COBALT/COBALT_diagnostics/Remapped_Data/Salt_woa_esm4grid.nc
Processing: /archive/cas/matlab_COBALT_diagnostics/Remapped_Data/phosphate_woa_esm4grid.mat
Variable: phosphate_woa
Writing out to: /work/jyl/proj/ESM4.2_COBALT/COBALT_diagnostics/Remapped_Data/phosphate_woa_esm4grid.nc
Processing: /archive/cas/matlab_COBALT_diagnostics/Remapped_Data/TCO2_glodap_esm4grid.mat
Variable: TCO2_glodap
Writing out to: /work/jyl/proj/ESM4.2_COBALT/COBALT_diagnostics/Remapped_Data/TCO2_glodap_esm4grid.nc
Processing: /archive/cas/matlab_COBALT_diagnostics/Remapped_Data/nitrate_woa_esm4grid.mat
Variable: nitrate_woa
Writing out to: /work/jyl/proj/ESM4.2_COBALT/COBALT_diagnostics/Remapped_Data/nitrate_woa_esm4grid.nc
Processing: /archive/cas/matlab_COBALT_diagnostics/Remapped_Data/silicate_woa_esm4grid.mat
Variable: silicate_woa
Writing out to: /work/jyl/proj/ES

In [ ]:
# file = '/archive/cas/matlab_COBALT_diagnostics/Remapped_Data/dco2_smoothed_monthclim_landschutzer_1992to2012_esm4grid.mat'
# varname_file = 'dco2_smoothed_monthclim_landschutzer_1992to2012_esm4grid'
# varname_out = 'dco2_smoothed_monthclim_landschutzer_1992to2012'

# data = loadmat(file)
# data = data[varname_file]
# data.shape

# if data.shape[0]==35:
#     da = xr.DataArray(data, coords={'z_l':esm4.z_l.values, 'yh':esm4.yh.values, 'xh':esm4.xh.values}, dims=['z_l','yh','xh'])
# if data.shape[0]==12:
#     da = xr.DataArray(data, coords={'time':np.arange(1,13), 'yh':esm4.yh.values, 'xh':esm4.xh.values}, dims=['time','yh','xh'])

# ds = da.to_dataset(name=varname_out)
# ds

# out_path='/work/jyl/proj/ESM4.2_COBALT/diagnostics/Remapped_Data/'
# filename=varname_file+'.nc'
# print(out_path+filename)
# ds.to_netcdf(out_path+filename, mode='w')